In [11]:
#!pip install depthai

In [25]:
from HandTrackerRenderer import HandTrackerRenderer
from HandTrackerEdge import HandTracker
from FPS import FPS, now
import numpy as np
import os
import cv2

In [26]:
pwd

'C:\\Users\\Ai\\Documents\\GitHub\\SeniorProject'

In [101]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Collected_Data') 
# Actions that we try to detect
actions = np.array(['and'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [102]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action))
        except:
            pass

In [103]:
def extractData(hands) :
    rh_data = np.zeros((21, 3))
    lh_data = np.zeros((21, 3))
    for hand in hands : 
        Label = hand.label
        if Label == 'left' : 
            lh_data = hand.norm_landmarks 
        elif Label == 'right' : 
            rh_data = hand.norm_landmarks
    return lh_data,rh_data
            

In [105]:

tracker = HandTracker(solo = False)

renderer = HandTrackerRenderer(
        tracker=tracker,
        output=None)

Palm detection blob     : C:\Users\Ai\Documents\GitHub\SeniorProject\models\palm_detection_sh4.blob
Landmark blob           : C:\Users\Ai\Documents\GitHub\SeniorProject\models\hand_landmark_lite_sh4.blob
PD post processing blob : C:\Users\Ai\Documents\GitHub\SeniorProject\custom_models\PDPostProcessing_top2_sh1.blob
Sensor resolution: (1920, 1080)
Internal camera FPS set to: 36
Internal camera image size: 1152 x 648 - pad_h: 252
Creating pipeline...
Creating Color Camera...
Creating Palm Detection pre processing image manip...
Creating Palm Detection Neural Network...
Creating Palm Detection post processing Neural Network...
Creating Hand Landmark pre processing image manip...
Creating Hand Landmark Neural Network (2 threads)...
Pipeline created.
Pipeline started - USB speed: SUPER


In [106]:
fps = FPS()
Break = False
for action in actions : 
    for i in range(no_sequences) :
        frame_num_array = []
        lh = []
        rh = [] 
        while True:
            # Run hand tracker on next frame
            # 'bag' contains some information related to the frame 
            # and not related to a particular hand like body keypoints in Body Pre Focusing mode
            # Currently 'bag' contains meaningful information only when Body Pre Focusing is used
            frame, hands, bag = tracker.next_frame()
            cv2.waitKey(100)
            frame_num = tracker.fps.nb_frames()
            frame_num_array.append(frame_num)
            frame_count = frame_num - frame_num_array[0]
            if frame is None: break
            if frame_count == 0 :  
                cv2.putText(frame, 'Start Collecting', (120,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.waitKey(1000)
            elif frame_count > 0  : 
                cv2.putText(frame,'Collecting frames for {},Video Number {}'.format(action,i),(15,30)
                                 , cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                lh_data,rh_data = extractData(hands)
                lh.append(lh_data)
                rh.append(rh_data)
            if frame_count >= sequence_length : 
                lrh = np.concatenate([lh,rh])
                npy_path = os.path.join(DATA_PATH, action,f"{action}_{i}")
                np.save(npy_path, lrh)
                break
            # Draw hands
            frame = renderer.draw(frame, hands, bag)
            key = renderer.waitKey(delay=1)
            if key == 27 or key == ord('q'):
                Break = True
                break
        renderer.exit()
        if Break : break
    if Break : break

    

In [ ]:
lh[0].shape

In [107]:
tracker.exit()